## Exercício: Modelo de Linguagem com auto-atenção e máscaras causais

Seguimos na mesma linha de treinar um modelo de linguagem a partir dos textos do livro "O Guarani", de José de Alencar.

Neste exercício, vamos treinar um modelo de linguagem com auto-atenção e com máscara causal. A máscara causal é necessária para que o modelo não tenha acesso a palavras futuras, que é a abordagem usada por grandes modelos de linguagem, como o GPT.

Use a implementação matricial de auto-atenção da aula passada.

### Modificações necessárias

* Adicione a máscara causal na função `forward` da cabeça de auto-atenção.
* Modifique o nosso dataloader para retornar inputs (uma lista de tokens de tamanho $n$), targets (uma lista de tokens de tamanho $n$ deslocada para a esquerda em 1 token). Exemplo `input = [1, 2, 3, 4]`, `target = [2, 3, 4, 5]` para a sequência `[1, 2, 3, 4, 5]` com `seq_len=4`, por exemplo (Ver slide 50).

### Extra
* MultiHeadAttention: modifique a cabeça de auto-atenção para ter múltiplas cabeças. Isso não é obrigatório, mas pode ser interessante para ver como o modelo se comporta.
* Diagrama da geração: fazer diagrama que mostre os passos da geração de tokens (conforme slide 47).

### Dicas

* Use como base o vídeo do Karpathy: https://www.youtube.com/watch?v=kCc8FmEb1nY. Observe que, no vídeo, ele primeiro implementa um modelo bi-grama, depois um modelo de linguagem com auto-atenção. O modelo de auto-atenção é implementado por volta do minuto 40, mas vale a pena assistir o vídeo todo.
* Use esta implementação como base: https://colab.research.google.com/drive/1vFTg4MSXVJwNSzPjaCcvmqhxTP7gK7HA?usp=sharing. Observe como o modelo é organizado e como a máscara é implementada na classe MultiHeadAttention.
* Use `context_size=9`

## Faz download e carrega o dataset

In [ ]:
!wget https://www.gutenberg.org/ebooks/67724.txt.utf-8
!wget https://www.gutenberg.org/ebooks/67725.txt.utf-8

--2024-03-06 23:38:57--  https://www.gutenberg.org/ebooks/67724.txt.utf-8
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://www.gutenberg.org/cache/epub/67724/pg67724.txt [following]
--2024-03-06 23:38:57--  http://www.gutenberg.org/cache/epub/67724/pg67724.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/cache/epub/67724/pg67724.txt [following]
--2024-03-06 23:38:58--  https://www.gutenberg.org/cache/epub/67724/pg67724.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 372908 (364K) [text/plain]
Saving to: ‘67724.txt.utf-8’

67724.txt.utf-8     100%[===================>] 364

In [ ]:
text = open("67724.txt.utf-8","r").read()
text += open("67725.txt.utf-8","r").read()

paragraphs = text.split("\n\n")
len(paragraphs)

4969

In [ ]:
cleaned_paragraphs = [paragraph.replace("\n", " ") for paragraph in paragraphs if paragraph.strip()]

"""TODO: Ver como tá ficando os paragrafos"""

len(cleaned_paragraphs)

4892

## Análise do dataset

In [ ]:
# Conta as palavras no dataset
from collections import Counter
import re

def count_words(texts):
    word_counts = Counter()
    for text in texts:
        word_counts.update(re.findall(r'\w+', text.lower()))
    return word_counts

word_counts = count_words(cleaned_paragraphs)

len(word_counts)

12603

## Criando um vocabulário

In [ ]:
vocab_size = 10000
most_frequent_words = [word for word, count in word_counts.most_common(vocab_size)]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)}

In [ ]:
def encode_sentence(sentence, vocab):
    return [vocab.get(word, 0) for word in re.findall(r'\w+', sentence.lower())]

encode_sentence(cleaned_paragraphs[20], vocab)

[6594,
 139,
 4376,
 19,
 6595,
 6,
 44,
 110,
 269,
 259,
 2662,
 10,
 1064,
 6596,
 2,
 186,
 130,
 280,
 3,
 2257,
 6,
 6597,
 1,
 2665]

## Classe do dataset

In [ ]:
context_size = 5 # 5 palavras de entrada. O target é a próxima palavra
"""TODO: Preparar o dataset"""

In [ ]:
"""TODO: divida o dataset em validação/treino com um proporção de 20/80 %. OBS, use random_state=18"""

In [ ]:
"""TODO: implemente a classe do dataset"""

train_data = MyDataset(...)
val_data = MyDataset

95377

In [ ]:
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
sample = next(iter(train_loader))

## Model

In [ ]:
import torch.nn as nn

class LanguageModel(torch.nn.Module):
    """TODO: implementar o modelo de linguagem"""

In [ ]:
model = ...

In [ ]:
# sample = next(iter(train_loader))
input = sample[0]
target = sample[1]

In [ ]:
output = model(input)

In [ ]:
output.argmax(dim=1)

tensor([4842, 2163, 7516, 2652, 6373, 7429, 8003, 3759, 1768, 7740, 2595, 1859,
        3189, 8049, 5727, 6132])

In [ ]:
target

tensor([   2,    3,    4,   37,    3,  215,   71,  411, 1263,  355,   87, 3653,
         584,  980,    1,    7])

## Training

In [ ]:
# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
epochs = 10
lr = """TODO""""
criterion = """TODO CrossEntropy""""

optimizer = """TODO: AdamW ou outro""""

model.to(device)

"""TODO: Implemente o loop de treinamento. Em cada época, calcule e imprima a loss no dataset de validação""""

## Avaliação

In [ ]:
""" TODO: calcule a perplexidade final no dataset de validação """

## Exemplo de uso

In [ ]:
text = ""

def generate_text(model, vocab, text, max_length):
    """TODO: implemente a função para gerar texto até atingir o max_length"""

context = 5
max_length= 10
generate_text(text, max_length)